In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.layers import Dropout
from keras.regularizers import l2, l1
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from keras.constraints import MaxNorm
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve
import seaborn as sns
import tensorflowjs as tfjs

In [ ]:
train_dir = '/content/drive/MyDrive/Tensorflow/T3/train3'
test_dir = '/content/drive/MyDrive/Tensorflow/train'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# custom callback to calculate metrics after each fold
class MetricsCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs={}):
        y_pred = np.argmax(self.model.predict(self.validation_data[0]), axis=1)
        y_true = np.argmax(self.validation_data[1], axis=1)

        confusion = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        print(f'Confusion Matrix:\n{confusion}')
        print(f'Precision: {precision}')
        print(f'Recall: {recall}')
        print(f'F1 Score: {f1}')
        metrics_callback = MetricsCallback(validation_data=val_generator)
class LearningRateLogger(Callback):
    def on_train_begin(self, logs=None):
        self.lrates = []

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        self.lrates.append(lr)

lr_logger = LearningRateLogger()

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
# predictions = Dense(2, activation='softmax')(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_reducer = ReduceLROnPlateau(factor=0.5, min_lr=1e-11)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_reducer, lr_logger]
)

In [ ]:
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())
total_test_samples = test_generator.samples

test_true_labels = []
for i in range(total_test_samples):
    label_index = test_generator.labels[i]
    test_true_labels.append(label_index)

test_accuracy_list = history.history['val_accuracy']
precision_list_DR = []
recall_list_DR = []
f1_list_DR = []
precision_list_NO_DR = []
recall_list_NO_DR = []
f1_list_NO_DR = []

for epoch in range(1, len(test_accuracy_list) + 1):
    test_pred_probs = model.predict(test_generator)
    test_pred_labels = np.argmax(test_pred_probs, axis=1)

    # metrics for DR class
    precision_DR = precision_score(test_true_labels, test_pred_labels, pos_label=class_names.index('DR'))
    recall_DR = recall_score(test_true_labels, test_pred_labels, pos_label=class_names.index('DR'))
    f1_DR = f1_score(test_true_labels, test_pred_labels, pos_label=class_names.index('DR'))
    precision_list_DR.append(precision_DR)
    recall_list_DR.append(recall_DR)
    f1_list_DR.append(f1_DR)

    # metrics for NO_DR class
    precision_NO_DR = precision_score(test_true_labels, test_pred_labels, pos_label=class_names.index('NO_DR'))
    recall_NO_DR = recall_score(test_true_labels, test_pred_labels, pos_label=class_names.index('NO_DR'))
    f1_NO_DR = f1_score(test_true_labels, test_pred_labels, pos_label=class_names.index('NO_DR'))
    precision_list_NO_DR.append(precision_NO_DR)
    recall_list_NO_DR.append(recall_NO_DR)
    f1_list_NO_DR.append(f1_NO_DR)

In [ ]:
# graph of training accuracy and validation accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
precision, recall, _ = precision_recall_curve(test_true_labels, test_pred_labels)

plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
#confusion matrix
cm = confusion_matrix(test_true_labels, test_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_pred_probs = model.predict(test_generator)
test_pred_labels = np.argmax(test_pred_probs, axis=1)

test_true_labels = test_generator.classes

precision = precision_score(test_true_labels, test_pred_labels)
recall = recall_score(test_true_labels, test_pred_labels)
f1 = f1_score(test_true_labels, test_pred_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
target_directory = '/content/drive/MyDrive/Tensorflow/T3/ts-js/'
tfjs.converters.save_keras_model(model, target_directory)